In [1]:
import h5py
import pandas as pd


# dfg.to_hdf('dfg3_4.h5', key='losses')

dfg = pd.read_hdf('dfg3_4.h5', key='losses')
dfg

,σ^v_[kPa],PI,LL,wn,OCR,Cu-Tx
3,27.000,41.090909,66.545455,29.200000,5.730159,68.744449
4,27.000,43.454545,69.727273,29.600000,5.730159,68.744449
5,27.000,45.818182,72.909091,30.000000,5.730159,68.744449
6,28.000,48.181818,76.090909,30.000000,5.730159,68.744449
7,31.500,50.545455,79.272727,30.000000,5.730159,68.744449
...,...,...,...,...,...,...
460,315.875,50.000000,76.000000,41.933333,2.009089,221.478496
461,316.750,50.000000,76.000000,42.966667,2.018177,222.835221
462,317.000,50.000000,76.000000,44.000000,2.027266,224.191945
463,317.500,49.600000,77.466667,42.000000,2.036354,225.548670


In [2]:
import scipy.stats as st
import statsmodels as sm
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings

matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
#matplotlib.style.use('ggplot')

# Create models from data
def best_fit_distribution(data, bins=10, ax=None):
    """Model data by finding best fit distribution to data"""
    # Get histogram of original data
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0

    # Distributions to check
    DISTRIBUTIONS_ALL = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy]
    
    # Distributions available in pymc3
    DISTRIBUTIONS = [    
        st.uniform, st.norm, st.halfnorm, st.gamma, st.expon, st.cauchy]
    
    # Distributions available in pymc3
#     DISTRIBUTIONS = [st.norm]


    # Best holders
    best_distribution = st.norm
    best_params = (0.0, 1.0)
    best_sse = np.inf

    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:

        # Try to fit the distribution
        try:
            # Ignore warnings from data that can't be fit
            with warnings.catch_warnings():
                warnings.filterwarnings('ignore')

                # fit dist to data
                params = distribution.fit(data)

                # Separate parts of parameters
                arg = params[:-2]
                loc = params[-2]
                scale = params[-1]

                # Calculate fitted PDF and error with fit in distribution
                pdf = distribution.pdf(x, loc=loc, scale=scale, *arg)
                sse = np.sum(np.power(y - pdf, 2.0))

                # if axis pass in add to plot
                
                try:
                    if ax:
                        pd.Series(pdf, x).plot(ax=ax)
                    pass

                except Exception as ex:
                    
                    template = "An exception of type {0} occurred. Arguments:\n{1!r}"
                    message = template.format(type(ex).__name__, ex.args)
                    print(message)
                    
                    pass



                # identify if this distribution is better
                if best_sse > sse > 0:
                    best_distribution = distribution
                    best_params = params
                    best_sse = sse


        except Exception as ex:
            
            template = "An exception of type {0} occurred. Arguments:\n{1!r}"
            message = template.format(type(ex).__name__, ex.args)
            print(message)
            
            pass

    return (best_distribution.name, best_params)

def make_pdf(dist, params, size=10000):
    """Generate distributions's Probability Distribution Function """

    # Separate parts of parameters
    arg = params[:-2]
    loc = params[-2]
    scale = params[-1]

    # Get sane start and end points of distribution
    start = dist.ppf(0.01, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.01, loc=loc, scale=scale)
    end = dist.ppf(0.99, *arg, loc=loc, scale=scale) if arg else dist.ppf(0.99, loc=loc, scale=scale)

    # Build PDF and turn into pandas Series
    x = np.linspace(start, end, size)
    y = dist.pdf(x, loc=loc, scale=scale, *arg)
    pdf = pd.Series(y, x)

    return pdf

In [1]:
import numpy as  np
# Load data from statsmodels datasets

#data = pd.Series(sm.datasets.elnino.load_pandas().data.set_index('YEAR').values.ravel())

n_cols = dfg.shape[1]

par_dict={}

for e in np.arange(0,n_cols,1):
    
    print('start '+str(dfg.iloc[:,e].name))

    data = dfg.iloc[:,e].dropna().reset_index(drop=True).copy()
    data = pd.Series(data.values.ravel())
    
    bins =100
    
    y, x = np.histogram(data, bins=bins, density=True)
    x = (x + np.roll(x, -1))[:-1] / 2.0


    # Plot for comparison
    plt.figure(figsize=(12,8))
    ax = data.plot(kind='hist', bins=100, density=True, alpha=0.5)
    # Save plot limits
    dataYLim = ax.get_ylim()

    # Find best fit distribution
    best_fit_name, best_fit_params = best_fit_distribution(data, 100, ax)
    best_dist = getattr(st, best_fit_name)
    
    print('Distribution found: '+str(best_fit_name))

    # Update plots
    ax.set_ylim(dataYLim)
    ax.set_title(dfg.columns[e] + u'\n All Fitted Distributions')
    ax.set_xlabel('')
    ax.set_ylabel('Frequency')

    # Make PDF with best params 
    pdf = make_pdf(best_dist, best_fit_params)

    # Display
    plt.figure(figsize=(12,8))
    ax = pdf.plot(lw=2, label='PDF', legend=True)
    data.plot(kind='hist', bins=50, density=True, alpha=0.5, label='Data', legend=True, ax=ax)

    param_names = (best_dist.shapes + ', loc, scale').split(', ') if best_dist.shapes else ['loc', 'scale']
    param_str = ', '.join(['{}={:0.2f}'.format(k,v) for k,v in zip(param_names, best_fit_params)])
    dist_str = '{}({})'.format(best_fit_name, param_str)
    
    # export a list  "par_list" with all results
    
    dict
    par_list = ([(k,v) for k,v in zip(param_names, best_fit_params)])
    par_dict[dfg.columns[e] + '_'+best_fit_name] = par_list

    ax.set_title(dfg.columns[e] + u'\n' + dist_str)
    ax.set_xlabel('')
    ax.set_ylabel('Frequency')

NameError: name 'dfg' is not defined

In [4]:
DISTRIBUTIONS = [        
        st.alpha,st.anglit,st.arcsine,st.beta,st.betaprime,st.bradford,st.burr,st.cauchy,st.chi,st.chi2,st.cosine,
        st.dgamma,st.dweibull,st.erlang,st.expon,st.exponnorm,st.exponweib,st.exponpow,st.f,st.fatiguelife,st.fisk,
        st.foldcauchy,st.foldnorm,st.frechet_r,st.frechet_l,st.genlogistic,st.genpareto,st.gennorm,st.genexpon,
        st.genextreme,st.gausshyper,st.gamma,st.gengamma,st.genhalflogistic,st.gilbrat,st.gompertz,st.gumbel_r,
        st.gumbel_l,st.halfcauchy,st.halflogistic,st.halfnorm,st.halfgennorm,st.hypsecant,st.invgamma,st.invgauss,
        st.invweibull,st.johnsonsb,st.johnsonsu,st.ksone,st.kstwobign,st.laplace,st.levy,st.levy_l,st.levy_stable,
        st.logistic,st.loggamma,st.loglaplace,st.lognorm,st.lomax,st.maxwell,st.mielke,st.nakagami,st.ncx2,st.ncf,
        st.nct,st.norm,st.pareto,st.pearson3,st.powerlaw,st.powerlognorm,st.powernorm,st.rdist,st.reciprocal,
        st.rayleigh,st.rice,st.recipinvgauss,st.semicircular,st.t,st.triang,st.truncexpon,st.truncnorm,st.tukeylambda,
        st.uniform,st.vonmises,st.vonmises_line,st.wald,st.weibull_min,st.weibull_max,st.wrapcauchy]

In [11]:
best_fit_params

(4.578282604721661, -41.31285768850843, 51.888490083704056)

In [8]:
par_dict

{'σ^v_[kPa]_gamma': [('a', 5.7484772667104025),
  ('loc', -79.74401154567104),
  ('scale', 49.55961974402014)],
 'PI_cauchy': [('loc', 48.13315394405227), ('scale', 25.31411714292492)],
 'LL_cauchy': [('loc', 78.09132799206728), ('scale', 27.017437148764202)],
 'wn_cauchy': [('loc', 41.95689437086652), ('scale', 19.78169219769638)],
 'OCR_gamma': [('a', 0.4117660245870177),
  ('loc', 1.9209220425804383),
  ('scale', 82.32680334284095)],
 'Cu-Tx_gamma': [('a', 4.578282604721661),
  ('loc', -41.31285768850843),
  ('scale', 51.888490083704056)]}

In [10]:
import pickle

file = open('par_dict.txt', 'w')
pickle.dump(par_dict, file)
file.close()

TypeError: write() argument must be str, not bytes